In [1]:
import pandas as pd
import scipy.stats as ss
import numpy.typing as npt
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Iterable, Tuple
import re
import pathlib
import bioframe as bf
import scipy.stats as ss
from statsmodels.stats.multitest import fdrcorrection

In [2]:
# See overlap_subcomps_with_de_genes.py


def get_subcompartment_ranks() -> dict:
    compartment_labels = tuple(["B3", "B2", "B1", "B0", "A0", "A1", "A2", "A3"])
    return {k: v for v, k in enumerate(compartment_labels)}


def import_subcomps(path_to_bedgraph: pathlib.Path, cond1: str, cond2: str) -> pd.DataFrame:
    """
    Read a bedgraph into a df with the following columns:
    chrom, start, end, padj, *.state
    where *.state stores the subcompartment label for the given interval.
    """
    df = pd.read_table(path_to_bedgraph).rename(columns={"chr": "chrom"}).set_index(["chrom", "start", "end"])
    for cond in (cond1, cond2):
        if f"{cond}.state" not in df:
            raise RuntimeError(f"{cond} not found in {path_to_bedgraph}")
    return df.filter(regex=".state$").rename(columns=lambda c: c.removesuffix(".state"))[[cond1, cond2]]


def extract_attribute_gtf(data: pd.Series, key: str) -> List[str]:
    pattern = re.compile(rf"{key} \"(.*?)\";")

    return data.str.extract(pattern)


def import_gtf(path_to_gtf: pathlib.Path) -> pd.DataFrame:
    df = bf.read_table(path_to_gtf, schema="gtf", comment="#")
    df = df[df["feature"] == "gene"]

    for key in ["gene_id", "gene_name", "gene_type"]:
        df[key] = extract_attribute_gtf(df["attributes"], key)

    return df.drop(columns="attributes").set_index("gene_id").sort_index()


def extract_gene_types(dfs: Iterable[pd.DataFrame], gene_type_key="gene_type"):
    return np.sort(np.unique(list(itertools.chain.from_iterable((df[gene_type_key].unique() for df in dfs)))))


def import_de_gene_table(path_to_tsv: pathlib.Path, contrast: str, condition: str) -> pd.DataFrame:
    df = pd.read_table(path_to_tsv).set_index("id")
    df.index.name = "gene_id"
    assert df["contrast"].isin([contrast]).any()
    assert df["condition"].isin([condition]).any()

    return df[(df["contrast"] == contrast) & (df["condition"] == condition)].drop(columns=["contrast", "condition"])


def overlap_sucompartments_with_tss(subcomps: pd.DataFrame, de_table: pd.DataFrame, gtf: pd.DataFrame):
    gtf1 = gtf.copy().reset_index()
    gtf1["end"] = gtf1["start"] + 1
    df = (
        bf.overlap(subcomps.reset_index(), gtf1, how="inner", suffixes=("_", ""))
        .drop(columns=["chrom_", "start_", "end_"])
        .rename(columns=lambda c: c.rstrip("_"))
        .set_index("gene_id")
    )
    return de_table.merge(df, left_index=True, right_index=True)


def build_pvalue_matrix_upreg(df: pd.DataFrame, lfc_cutoff: float) -> npt.NDArray:
    num_comps = len(get_subcompartment_ranks())
    m = np.full([num_comps, num_comps], np.nan, dtype=float)

    for s1, i1 in get_subcompartment_ranks().items():
        for s2, i2 in get_subcompartment_ranks().items():
            if i2 == i1:
                continue

            dff = df[(df["contrast"].isin({s1, s2})) & (df["condition"].isin({s1, s2}))]

            if i1 > i2:
                df1 = dff[(dff["log2FoldChange"] >= lfc_cutoff) & (dff["delta"] < 0)]
            else:
                df1 = dff[(dff["log2FoldChange"] >= lfc_cutoff) & (dff["delta"] > 0)]

            df2 = dff[(dff["log2FoldChange"] >= lfc_cutoff) & (dff["delta"] != 0)]

            if len(df2) == 0:
                continue

            res = ss.binomtest(len(df1), len(df2), alternative="greater")
            m[i1, i2] = res.pvalue

    return m


def build_pvalue_matrix_downreg(df: pd.DataFrame, lfc_cutoff: float) -> npt.NDArray:
    num_comps = len(get_subcompartment_ranks())
    m = np.full([num_comps, num_comps], np.nan, dtype=float)

    for s1, i1 in get_subcompartment_ranks().items():
        for s2, i2 in get_subcompartment_ranks().items():
            if i2 == i1:
                continue

            dff = df[(df["contrast"].isin({s1, s2})) & (df["condition"].isin({s1, s2}))]

            if i1 > i2:
                df1 = dff[(dff["log2FoldChange"] <= -lfc_cutoff) & (dff["delta"] < 0)]
            else:
                df1 = dff[(dff["log2FoldChange"] <= -lfc_cutoff) & (dff["delta"] > 0)]

            df2 = dff[(dff["log2FoldChange"] <= -lfc_cutoff) & (dff["delta"] != 0)]

            if len(df2) == 0:
                continue

            res = ss.binomtest(len(df1), len(df2), alternative="greater")
            m[i1, i2] = res.pvalue

    return m


def compute_fisher_exact_test(df: pd.DataFrame, lfc_cutoff: float) -> Tuple[pd.DataFrame, float, float]:
    df["delta"] = df["condition"].map(get_subcompartment_ranks()) - df["contrast"].map(get_subcompartment_ranks())

    m = np.zeros([2, 2], dtype=int)
    m[0][0] = len(df[(df["log2FoldChange"] >= lfc_cutoff) & (df["delta"] < 0)])
    m[0][1] = len(df[(df["log2FoldChange"] >= lfc_cutoff) & (df["delta"] > 0)])

    m[1][0] = len(df[(df["log2FoldChange"] <= -lfc_cutoff) & (df["delta"] < 0)])
    m[1][1] = len(df[(df["log2FoldChange"] <= -lfc_cutoff) & (df["delta"] > 0)])

    df = pd.DataFrame(
        m,
        columns=["delta < 0", "delta > 0"],
        index=[f"lfc >= {lfc_cutoff:g}", f"lfc <= {-lfc_cutoff:g}"],
    )

    return df, *ss.fisher_exact(m)

In [3]:
df1 = (
    overlap_sucompartments_with_tss(
        import_subcomps(
            "../data/output/compartment_analysis/10000/MCF10A_WT_T1_C1_10000.subcompartments.bedGraph.gz",
            "MCF10A_WT",
            "MCF10A_T1",
        ),
        import_de_gene_table(
            "../data/output/diff_expression_analysis/star_salmon_gene/lfc_2.0/MCF10A_WT_vs_MCF10A_T1.tsv.gz",
            "MCF10A_WT",
            "MCF10A_T1",
        ),
        import_gtf("../data/input/hg38/hg38_gencode_v43.gtf.gz"),
    )
    .rename(columns={"gene_name_y": "gene_name"})
    .drop(columns=["gene_name_x"])
)


df2 = (
    overlap_sucompartments_with_tss(
        import_subcomps(
            "../data/output/compartment_analysis/10000/MCF10A_WT_T1_C1_10000.subcompartments.bedGraph.gz",
            "MCF10A_WT",
            "MCF10A_C1",
        ),
        import_de_gene_table(
            "../data/output/diff_expression_analysis/star_salmon_gene/lfc_2.0/MCF10A_WT_vs_MCF10A_C1.tsv.gz",
            "MCF10A_WT",
            "MCF10A_C1",
        ),
        import_gtf("../data/input/hg38/hg38_gencode_v43.gtf.gz"),
    )
    .rename(columns={"gene_name_y": "gene_name"})
    .drop(columns=["gene_name_x"])
)

gene_types = ["protein_coding", "lncRNA"]
padj_cutoff = 0.01
lfc_cutoff = 2.0

In [4]:
def matrix_to_df(m: npt.NDArray, contrast: str, condition: str) -> pd.DataFrame:
    data = []
    for s1, i1 in get_subcompartment_ranks().items():
        for s2, i2 in get_subcompartment_ranks().items():
            data.append([s1, s2, m[i1, i2]])

    df = pd.DataFrame(data, columns=["contrast", "condition", "pvalue"])
    df["pvalue-adjusted"] = fdrcorrection(df["pvalue"])[1]

    return df

In [5]:
contrast = "MCF10A_WT"
condition = "MCF10A_T1"

padj = "padj" if "padj" in df1 else "svalue"
mask = df1["gene_type"].str.lower().isin([s.lower() for s in gene_types])
de = df1[(df1[padj] <= padj_cutoff) & mask].rename(columns={condition: "condition", contrast: "contrast"})
de["delta"] = de["condition"].map(get_subcompartment_ranks()) - de["contrast"].map(get_subcompartment_ranks())

pv_downreg = matrix_to_df(
    np.nan_to_num(build_pvalue_matrix_downreg(de, lfc_cutoff), nan=1.0),
    contrast,
    condition,
)

pv_upreg = matrix_to_df(
    np.nan_to_num(build_pvalue_matrix_upreg(de, lfc_cutoff), nan=1.0),
    contrast,
    condition,
)


ctable, fstat, fpval = compute_fisher_exact_test(de, lfc_cutoff)

print(contrast, condition, fstat, fpval)

pv_downreg.to_csv(f"/tmp/{contrast}_{condition}_downreg.tsv", sep="\t", index=False)
pv_upreg.to_csv(f"/tmp/{contrast}_{condition}_upreg.tsv", sep="\t", index=False)
ctable.to_csv(f"/tmp/{contrast}_{condition}_fisher_ctable.tsv", sep="\t")

MCF10A_WT MCF10A_T1 0.2975206611570248 4.038076214854218e-09


In [6]:
contrast = "MCF10A_WT"
condition = "MCF10A_C1"

padj = "padj" if "padj" in df2 else "svalue"
mask = df2["gene_type"].str.lower().isin([s.lower() for s in gene_types])
de = df2[(df2[padj] <= padj_cutoff) & mask].rename(columns={condition: "condition", contrast: "contrast"})
de["delta"] = de["condition"].map(get_subcompartment_ranks()) - de["contrast"].map(get_subcompartment_ranks())

pv_downreg = np.nan_to_num(build_pvalue_matrix_downreg(de, lfc_cutoff), nan=1.0)
pv_upreg = np.nan_to_num(build_pvalue_matrix_upreg(de, lfc_cutoff), nan=1.0)

pv_downreg = matrix_to_df(
    np.nan_to_num(build_pvalue_matrix_downreg(de, lfc_cutoff), nan=1.0),
    contrast,
    condition,
)

pv_upreg = matrix_to_df(
    np.nan_to_num(build_pvalue_matrix_upreg(de, lfc_cutoff), nan=1.0),
    contrast,
    condition,
)

ctable, fstat, fpval = compute_fisher_exact_test(de, lfc_cutoff)

print(contrast, condition, fstat, fpval)

pv_downreg.to_csv(f"/tmp/{contrast}_{condition}_downreg.tsv", sep="\t", index=False)
pv_upreg.to_csv(f"/tmp/{contrast}_{condition}_upreg.tsv", sep="\t", index=False)
ctable.to_csv(f"/tmp/{contrast}_{condition}_fisher_ctable.tsv", sep="\t")

MCF10A_WT MCF10A_C1 0.4276178720527073 6.657467681675556e-13
